In [25]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import numpy as np
import regex as re
from bs4 import BeautifulSoup
import requests
import urllib
from urllib.request import urlopen
from selenium.common.exceptions import NoSuchElementException
from collections import Counter

In [26]:
Dataframe = pd.read_csv('Woo-5-mei.csv')

display(Dataframe[3040:])

,Titel,Verzoek,Specifiek,Verantwoordelijke,Thema,Documentsoort,Publicatiedatum,Documentdatum,Onderwerp,url,verzoekdatum
3040,Bijlagen deel 1 bij besluit op Woo-verzoek ove...,Bijlagen deel 1 bij besluit op Woo-verzoek ove...,Bijlagen 11-17 bij besluit op Woo-verzoek over...,Ministerie van Infrastructuur en Waterstaat,Water,Beslissing op Wob-/Woo-verzoek,06-03-2023,28-10-2022,NaN,https://open.overheid.nl/documenten/ronl-a624d...,NaN
3041,Besluit op Woo-verzoek over overeenkomst Arcad...,Besluit op Woo-verzoek over overeenkomst Arcadis,Lijst van documenten,Ministerie van Infrastructuur en Waterstaat,Water,Beslissing op Wob-/Woo-verzoek,06-03-2023,28-10-2022,NaN,https://open.overheid.nl/documenten/ronl-a77f8...,NaN
3042,Bijlagen deel 1 bij besluit op Woo-verzoek ove...,Bijlagen deel 1 bij besluit op Woo-verzoek ove...,Bijlage 20 bij besluit op Woo-verzoek over ove...,Ministerie van Infrastructuur en Waterstaat,Water,Beslissing op Wob-/Woo-verzoek,06-03-2023,28-10-2022,NaN,https://open.overheid.nl/documenten/ronl-a97fa...,NaN
3043,Bijlagen deel 1 bij besluit op Woo-verzoek ove...,Bijlagen deel 1 bij besluit op Woo-verzoek ove...,Bijlagen 22-23 bij besluit op Woo-verzoek over...,Ministerie van Infrastructuur en Waterstaat,Water,Beslissing op Wob-/Woo-verzoek,06-03-2023,28-10-2022,NaN,https://open.overheid.nl/documenten/ronl-c0708...,NaN
3044,Bijlagen deel 1 bij besluit op Woo-verzoek ove...,Bijlagen deel 1 bij besluit op Woo-verzoek ove...,Bijlagen 18-19 bij besluit op Woo-verzoek over...,Ministerie van Infrastructuur en Waterstaat,Water,Beslissing op Wob-/Woo-verzoek,06-03-2023,28-10-2022,NaN,https://open.overheid.nl/documenten/ronl-c9d13...,NaN
3045,Besluit op Woo-verzoek over meststoffen en afv...,Besluit op Woo-verzoek over meststoffen en afv...,NaN,"Ministerie van Landbouw, Natuur en Voedselkwal...",Stoffen,Beslissing op Wob-/Woo-verzoek,06-03-2023,03-03-2023,Mest,https://open.overheid.nl/documenten/ronl-cbaf4...,23-05-2022
3046,Besluit Woo-verzoek ECER-website - Openbaar ge...,Besluit Woo-verzoek ECER-website,Openbaar gemaakte documenten deel 1,Ministerie van Buitenlandse Zaken,Europese zaken,Beslissing op Wob-/Woo-verzoek,06-03-2023,08-02-2023,Europese Unie,https://open.overheid.nl/documenten/ronl-da28a...,NaN
3047,Besluit op Woo-verzoek over aanvraag tot verpl...,Besluit op Woo-verzoek over aanvraag tot verpl...,Woo-verzoek aanvraag verplichtstelling deelnem...,Ministerie van Sociale Zaken en Werkgelegenheid,"['Arbeidsverhoudingen', 'Arbeidsverhoudingen']",Beslissing op Wob-/Woo-verzoek,06-03-2023,03-03-2023,"['Arbeidsovereenkomst en cao', 'Pensioen']",https://open.overheid.nl/documenten/ronl-e1591...,19-05-2022
3048,Bijlagen deel 3 bij besluit op Woo-verzoek ove...,Bijlagen deel 3 bij besluit op Woo-verzoek ove...,Bijlagen 195-197 bij besluit op Woo-verzoek ov...,Ministerie van Infrastructuur en Waterstaat,Water,Beslissing op Wob-/Woo-verzoek,06-03-2023,28-10-2022,NaN,https://open.overheid.nl/documenten/ronl-e1644...,NaN
3049,Bijlagen deel 2 bij besluit op Woo-verzoek ove...,Bijlagen deel 2 bij besluit op Woo-verzoek ove...,Bijlagen 43-50 bij besluit op Woo-verzoek over...,Ministerie van Infrastructuur en Waterstaat,Water,Beslissing op Wob-/Woo-verzoek,06-03-2023,28-10-2022,NaN,https://open.overheid.nl/documenten/ronl-e36fc...,NaN


Deelbesluiten etc omzetten tot gewoon besluit. 

In [27]:
def eerstwrd(woord):
    
    eerste3woorden = woord.split(' ')[:3]
    string = ' '.join(eerste3woorden)
    
    if 'besluit' in string:
        if eerste3woorden[2].isnumeric() == True:
            Split = woord.split(eerste3woorden[2])
        elif eerste3woorden[1].isnumeric() == True:
            Split = woord.split(eerste3woorden[1])    
            
        else:
            Split = woord.split('besluit')
        
        return 'Besluit' + Split[1]
    
    elif 'beslissing' in string:
        
        if eerste3woorden[2].isnumeric() == True:
            Split = woord.split(eerste3woorden[2])
        elif eerste3woorden[1].isnumeric() == True:
            Split = woord.split(eerste3woorden[1])    
        else:
            Split = woord.split('beslissing')
            
        return 'Beslissing' + Split[1]
    
    else:
        return woord

In [28]:
Dataframe['Documentdatum'] = pd.to_datetime(Dataframe['Documentdatum'], format='%d-%m-%Y')
Dataframe = Dataframe.loc[Dataframe['Documentdatum'] >= '01-05-2022']

Dataframe['Verzoek'] = Dataframe['Verzoek'].map(lambda x:eerstwrd(x))

# deze functie zal het wisselende beginwoord proberen te veranderen naar Besluit of Beslissing
Dataframe['eerste_woord'] = Dataframe['Verzoek'].str.split(' ').str.get(0)

# Omdat het moeilijk is om te bepalen welk beginwoord wel of niet te droppen heb ik besloten enkel deze te behouden.
woord_list = ['Besluit', 'Wob-verzoek', 'Beslissing', 'Wob-besluit', 'Woo-verzoek','Woo-besluit']

Dataframe = Dataframe[Dataframe['eerste_woord'].isin(woord_list)]
Dataframe = Dataframe.drop('eerste_woord', axis=1)

display(Dataframe)

,Titel,Verzoek,Specifiek,Verantwoordelijke,Thema,Documentsoort,Publicatiedatum,Documentdatum,Onderwerp,url,verzoekdatum
0,Besluit op Wob-verzoek over aankoop van het sc...,Besluit op Wob-verzoek over aankoop van het sc...,NaN,"Ministerie van Onderwijs, Cultuur en Wetenschap",Cultuur,Beslissing op Wob-/Woo-verzoek,02-05-2022,2022-04-29,Kunst en cultuur,https://open.overheid.nl/documenten/ronl-0b4b7...,NaN
1,Besluit op Wob-verzoek over het op afstand pla...,Besluit op Wob-verzoek over het op afstand pla...,bijlagen bij Wob-verzoek over op afstand plaat...,Ministerie van Financiën,Onbekend thema,Beslissing op Wob-/Woo-verzoek,02-05-2022,2022-04-29,NaN,https://open.overheid.nl/documenten/ronl-232e7...,NaN
2,Besluit op Wob-verzoek over aankoop van het sc...,Besluit op Wob-verzoek over aankoop van het sc...,Inventarislijst Wob-verzoek over aankoop van s...,"Ministerie van Onderwijs, Cultuur en Wetenschap",Cultuur,Beslissing op Wob-/Woo-verzoek,02-05-2022,2022-04-29,Kunst en cultuur,https://open.overheid.nl/documenten/ronl-2af63...,NaN
3,Besluit op Wob-verzoek over aankoop van het sc...,Besluit op Wob-verzoek over aankoop van het sc...,NaN,"Ministerie van Onderwijs, Cultuur en Wetenschap",Cultuur,Beslissing op Wob-/Woo-verzoek,02-05-2022,2022-04-29,Kunst en cultuur,https://open.overheid.nl/documenten/ronl-3210e...,NaN
4,Besluit op Wob-verzoek correspondentie over st...,Besluit op Wob-verzoek correspondentie over st...,wobbesluit-staatssteunaspecten-bik.pdf,Ministerie van Financiën,Inkoop en beheer,Beslissing op Wob-/Woo-verzoek,02-05-2022,2022-04-29,Staatssteun,https://open.overheid.nl/documenten/ronl-3c18a...,09-07-2021
...,...,...,...,...,...,...,...,...,...,...,...
3056,Besluit op Woo-verzoek over kennisgeving onder...,Besluit op Woo-verzoek over kennisgeving onder...,Besluit,Ministerie van Infrastructuur en Waterstaat,Onbekend thema,Beslissing op Wob-/Woo-verzoek,07-03-2023,2023-02-02,NaN,https://open.overheid.nl/documenten/ronl-16bf3...,09-12-2022
3057,Besluit op Woo-verzoek over kennisgeving onder...,Besluit op Woo-verzoek over kennisgeving onder...,Bijlagen,Ministerie van Infrastructuur en Waterstaat,Onbekend thema,Beslissing op Wob-/Woo-verzoek,07-03-2023,2023-02-02,NaN,https://open.overheid.nl/documenten/ronl-21c18...,NaN
3058,Besluit op Woo-verzoek over lozingsvergunning ...,Besluit op Woo-verzoek over lozingsvergunning ...,NaN,Ministerie van Infrastructuur en Waterstaat,Water,Beslissing op Wob-/Woo-verzoek,07-03-2023,2023-03-01,NaN,https://open.overheid.nl/documenten/ronl-37873...,NaN
3059,Besluit op Woo-verzoek over uitgezonden percen...,Besluit op Woo-verzoek over uitgezonden percen...,NaN,Ministerie van Economische Zaken en Klimaat,"['Dienstensector', 'Media']",Beslissing op Wob-/Woo-verzoek,07-03-2023,2023-03-03,"['Telecommunicatie', 'Media en publieke omroep']",https://open.overheid.nl/documenten/ronl-b937a...,NaN


Enkel besluiten van ministeries hebben de structuur die bruikbaar is. 

In [29]:
def eerste_woord(string):
    spl = string.split(' ')
    if spl[0] != 'Ministerie':
        return np.nan
    else:
        return string

In [30]:
print(len(Dataframe))
Dataframe['Verantwoordelijke'] = Dataframe['Verantwoordelijke'].map(lambda x:eerste_woord(x))
Dataframe = Dataframe[Dataframe['Verantwoordelijke'].notna()]
print(len(Dataframe))

2788
2775


Statistieken over de verzoekdatum:

In [31]:
x = Dataframe['verzoekdatum'].values.tolist()

total = len(x)
nans = len([i for i in x if i is np.nan])
imgs = len([i for i in x if i == 'image'])

print(f'{total} Totaal aantal documenten')
print(f"{nans} Aantal documenten zonder vindbare verzoekdatum")
print(f'{imgs} Documenten die niet leesbaar zijn (images)')

dates = total - nans - imgs
print(str((imgs/total) * 100) + ' % fotos')
print(str((dates/total) * 100) + ' % met verzoekdatum')

2775 Totaal aantal documenten
1922 Aantal documenten zonder vindbare verzoekdatum
315 Documenten die niet leesbaar zijn (images)
11.351351351351353 % fotos
19.387387387387385 % met verzoekdatum


Ik haal op dit moment alle besluiten met meerdere data weg maar er kan in de toekomst gekozen worden om dan de eerste of laatste datum te nemen afhankelijk van wat wenselijk is.

In [34]:
df_urls = Dataframe.groupby(['Verzoek'])['url'].apply(list).reset_index(name='urls')
# deelbesluiten hebben mogelijk meerdere data dus ook deze nemen we niet of later pas mee!
df_documentdatums = Dataframe.groupby(['Verzoek'])['Documentdatum'].apply(set).reset_index(name='docdatums')
df_documentdatums = df_documentdatums[df_documentdatums['docdatums'].map(lambda x: len(x)) == 1]
df_documentdatums['docdatums'] = df_documentdatums['docdatums'].apply(lambda x: list(x)[0])

df_verzoekdatums = Dataframe.groupby(['Verzoek'])['verzoekdatum'].apply(list).reset_index(name='verdatums')
df_verantwoordelijke = Dataframe.groupby(['Verzoek'])['Verantwoordelijke'].apply(set).reset_index(name='verantwoordelijke')
df_verantwoordelijke['verantwoordelijke'] = df_verantwoordelijke['verantwoordelijke'].apply(lambda x: list(x)[0])

display(df_urls)
display(df_documentdatums)
display(df_verzoekdatums)

,Verzoek,urls
0,Beslissing op bezwaar Wob verzoek corresponden...,[https://open.overheid.nl/documenten/ronl-34e6...
1,Beslissing op bezwaar Wob-/Woo-verzoek over in...,[https://open.overheid.nl/documenten/ronl-7a0f...
2,Beslissing op bezwaar Wob-/Woo-verzoek over ve...,[https://open.overheid.nl/documenten/ronl-0ba9...
3,Beslissing op bezwaar Wob-besluit Financiën ov...,[https://open.overheid.nl/documenten/ronl-e3e1...
4,Beslissing op bezwaar Wob-besluit over afstemm...,[https://open.overheid.nl/documenten/ronl-07e2...
...,...,...
1128,Besluit van Wob-verzoek over productverbetering,[https://open.overheid.nl/documenten/ronl-0491...
1129,Woo-verzoek Evaluatiecommissie Tuitjenhorn,[https://open.overheid.nl/documenten/ronl-370d...
1130,Woo-verzoek over de identiteit achter anonieme...,[https://open.overheid.nl/documenten/ronl-2a0b...
1131,Woo-verzoek over landbouwvrijstelling in inkom...,[https://open.overheid.nl/documenten/ronl-1c09...


,Verzoek,docdatums
0,Beslissing op bezwaar Wob verzoek corresponden...,2022-09-15
1,Beslissing op bezwaar Wob-/Woo-verzoek over in...,2022-12-23
2,Beslissing op bezwaar Wob-/Woo-verzoek over ve...,2023-01-27
3,Beslissing op bezwaar Wob-besluit Financiën ov...,2022-04-26
4,Beslissing op bezwaar Wob-besluit over afstemm...,2023-02-09
...,...,...
1127,Besluit van 20 juli 2022 op het verzoek inzake...,2022-07-20
1129,Woo-verzoek Evaluatiecommissie Tuitjenhorn,2022-06-30
1130,Woo-verzoek over de identiteit achter anonieme...,2022-08-25
1131,Woo-verzoek over landbouwvrijstelling in inkom...,2023-02-27


,Verzoek,verdatums
0,Beslissing op bezwaar Wob verzoek corresponden...,"[nan, nan, nan]"
1,Beslissing op bezwaar Wob-/Woo-verzoek over in...,"[25-07-2022, nan, nan]"
2,Beslissing op bezwaar Wob-/Woo-verzoek over ve...,"[image, 02-11-2021, nan]"
3,Beslissing op bezwaar Wob-besluit Financiën ov...,"[nan, nan]"
4,Beslissing op bezwaar Wob-besluit over afstemm...,"[nan, nan]"
...,...,...
1128,Besluit van Wob-verzoek over productverbetering,"[nan, nan]"
1129,Woo-verzoek Evaluatiecommissie Tuitjenhorn,"[22-10-2021, nan]"
1130,Woo-verzoek over de identiteit achter anonieme...,[02-08-2022]
1131,Woo-verzoek over landbouwvrijstelling in inkom...,"[nan, 07-08-2022, nan]"


In [35]:
display(df_verantwoordelijke)

,Verzoek,verantwoordelijke
0,Beslissing op bezwaar Wob verzoek corresponden...,Ministerie van Infrastructuur en Waterstaat
1,Beslissing op bezwaar Wob-/Woo-verzoek over in...,Ministerie van Infrastructuur en Waterstaat
2,Beslissing op bezwaar Wob-/Woo-verzoek over ve...,"Ministerie van Landbouw, Natuur en Voedselkwal..."
3,Beslissing op bezwaar Wob-besluit Financiën ov...,Ministerie van Financiën
4,Beslissing op bezwaar Wob-besluit over afstemm...,Ministerie van Financiën
...,...,...
1128,Besluit van Wob-verzoek over productverbetering,"Ministerie van Volksgezondheid, Welzijn en Sport"
1129,Woo-verzoek Evaluatiecommissie Tuitjenhorn,Ministerie van Justitie en Veiligheid
1130,Woo-verzoek over de identiteit achter anonieme...,"Ministerie van Landbouw, Natuur en Voedselkwal..."
1131,Woo-verzoek over landbouwvrijstelling in inkom...,Ministerie van Financiën


In [36]:
df_documentdatums = df_documentdatums.join(df_urls.set_index('Verzoek'), on='Verzoek')
df = df_documentdatums.join(df_verzoekdatums.set_index('Verzoek'), on='Verzoek')
df = df.join(df_verantwoordelijke.set_index('Verzoek'), on='Verzoek')

In [37]:
display(df)

,Verzoek,docdatums,urls,verdatums,verantwoordelijke
0,Beslissing op bezwaar Wob verzoek corresponden...,2022-09-15,[https://open.overheid.nl/documenten/ronl-34e6...,"[nan, nan, nan]",Ministerie van Infrastructuur en Waterstaat
1,Beslissing op bezwaar Wob-/Woo-verzoek over in...,2022-12-23,[https://open.overheid.nl/documenten/ronl-7a0f...,"[25-07-2022, nan, nan]",Ministerie van Infrastructuur en Waterstaat
2,Beslissing op bezwaar Wob-/Woo-verzoek over ve...,2023-01-27,[https://open.overheid.nl/documenten/ronl-0ba9...,"[image, 02-11-2021, nan]","Ministerie van Landbouw, Natuur en Voedselkwal..."
3,Beslissing op bezwaar Wob-besluit Financiën ov...,2022-04-26,[https://open.overheid.nl/documenten/ronl-e3e1...,"[nan, nan]",Ministerie van Financiën
4,Beslissing op bezwaar Wob-besluit over afstemm...,2023-02-09,[https://open.overheid.nl/documenten/ronl-07e2...,"[nan, nan]",Ministerie van Financiën
...,...,...,...,...,...
1127,Besluit van 20 juli 2022 op het verzoek inzake...,2022-07-20,[https://open.overheid.nl/documenten/ronl-7517...,[image],Ministerie van Justitie en Veiligheid
1129,Woo-verzoek Evaluatiecommissie Tuitjenhorn,2022-06-30,[https://open.overheid.nl/documenten/ronl-370d...,"[22-10-2021, nan]",Ministerie van Justitie en Veiligheid
1130,Woo-verzoek over de identiteit achter anonieme...,2022-08-25,[https://open.overheid.nl/documenten/ronl-2a0b...,[02-08-2022],"Ministerie van Landbouw, Natuur en Voedselkwal..."
1131,Woo-verzoek over landbouwvrijstelling in inkom...,2023-02-27,[https://open.overheid.nl/documenten/ronl-1c09...,"[nan, 07-08-2022, nan]",Ministerie van Financiën


In [39]:
verdatums = []
for verzoek in df['verdatums']:
    if type(verzoek) is not list:
        verdatums.append(verzoek)
    
    else:    
        datum = [i for i in verzoek if '-' in str(i)]
        if any(datum) == True:
            verdatums.append(datum[0])
        else:
            verdatums.append(verzoek)

df['verdatums'] = verdatums

display(df)

,Verzoek,docdatums,urls,verdatums,verantwoordelijke
0,Beslissing op bezwaar Wob verzoek corresponden...,2022-09-15,[https://open.overheid.nl/documenten/ronl-34e6...,"[nan, nan, nan]",Ministerie van Infrastructuur en Waterstaat
1,Beslissing op bezwaar Wob-/Woo-verzoek over in...,2022-12-23,[https://open.overheid.nl/documenten/ronl-7a0f...,25-07-2022,Ministerie van Infrastructuur en Waterstaat
2,Beslissing op bezwaar Wob-/Woo-verzoek over ve...,2023-01-27,[https://open.overheid.nl/documenten/ronl-0ba9...,02-11-2021,"Ministerie van Landbouw, Natuur en Voedselkwal..."
3,Beslissing op bezwaar Wob-besluit Financiën ov...,2022-04-26,[https://open.overheid.nl/documenten/ronl-e3e1...,"[nan, nan]",Ministerie van Financiën
4,Beslissing op bezwaar Wob-besluit over afstemm...,2023-02-09,[https://open.overheid.nl/documenten/ronl-07e2...,"[nan, nan]",Ministerie van Financiën
...,...,...,...,...,...
1127,Besluit van 20 juli 2022 op het verzoek inzake...,2022-07-20,[https://open.overheid.nl/documenten/ronl-7517...,[image],Ministerie van Justitie en Veiligheid
1129,Woo-verzoek Evaluatiecommissie Tuitjenhorn,2022-06-30,[https://open.overheid.nl/documenten/ronl-370d...,22-10-2021,Ministerie van Justitie en Veiligheid
1130,Woo-verzoek over de identiteit achter anonieme...,2022-08-25,[https://open.overheid.nl/documenten/ronl-2a0b...,02-08-2022,"Ministerie van Landbouw, Natuur en Voedselkwal..."
1131,Woo-verzoek over landbouwvrijstelling in inkom...,2023-02-27,[https://open.overheid.nl/documenten/ronl-1c09...,07-08-2022,Ministerie van Financiën


In [40]:
df_dates = df[df['verdatums'].apply(lambda x: isinstance(x, str))]
display(df_dates)

,Verzoek,docdatums,urls,verdatums,verantwoordelijke
1,Beslissing op bezwaar Wob-/Woo-verzoek over in...,2022-12-23,[https://open.overheid.nl/documenten/ronl-7a0f...,25-07-2022,Ministerie van Infrastructuur en Waterstaat
2,Beslissing op bezwaar Wob-/Woo-verzoek over ve...,2023-01-27,[https://open.overheid.nl/documenten/ronl-0ba9...,02-11-2021,"Ministerie van Landbouw, Natuur en Voedselkwal..."
17,Beslissing op bezwaar Woo-besluit over vraag 4...,2022-12-01,[https://open.overheid.nl/documenten/ronl-1534...,03-12-2021,Ministerie van Financiën
24,Beslissing op bezwaar op besluit Wob over VOC-...,2022-10-17,[https://open.overheid.nl/documenten/ronl-201b...,10-09-2021,"Ministerie van Landbouw, Natuur en Voedselkwal..."
27,Beslissing op bezwaar tegen Woo-besluit over A...,2023-02-02,[https://open.overheid.nl/documenten/ronl-f944...,09-08-2022,Ministerie van Buitenlandse Zaken
...,...,...,...,...,...
1124,Besluit over het corrigeren van onvolkomenhede...,2023-01-05,[https://open.overheid.nl/documenten/ronl-29d6...,14-10-2022,"Ministerie van Landbouw, Natuur en Voedselkwal..."
1129,Woo-verzoek Evaluatiecommissie Tuitjenhorn,2022-06-30,[https://open.overheid.nl/documenten/ronl-370d...,22-10-2021,Ministerie van Justitie en Veiligheid
1130,Woo-verzoek over de identiteit achter anonieme...,2022-08-25,[https://open.overheid.nl/documenten/ronl-2a0b...,02-08-2022,"Ministerie van Landbouw, Natuur en Voedselkwal..."
1131,Woo-verzoek over landbouwvrijstelling in inkom...,2023-02-27,[https://open.overheid.nl/documenten/ronl-1c09...,07-08-2022,Ministerie van Financiën


In [55]:
df_dates['docdatums'] = pd.to_datetime(df_dates['docdatums'], format = '%d-%m-%Y')
df_dates['verdatums'] = pd.to_datetime(df_dates['verdatums'], format = '%d-%m-%Y')


df_dates['dagen'] = (df_dates['docdatums'] - df_dates['verdatums']).dt.days
df_dates = df_dates[df_dates['dagen'] > 0]

display(df_dates)

,Verzoek,docdatums,urls,verdatums,verantwoordelijke,dagen
1,Beslissing op bezwaar Wob-/Woo-verzoek over in...,2022-12-23,[https://open.overheid.nl/documenten/ronl-7a0f...,2022-07-25,Ministerie van Infrastructuur en Waterstaat,151
2,Beslissing op bezwaar Wob-/Woo-verzoek over ve...,2023-01-27,[https://open.overheid.nl/documenten/ronl-0ba9...,2021-11-02,"Ministerie van Landbouw, Natuur en Voedselkwal...",451
17,Beslissing op bezwaar Woo-besluit over vraag 4...,2022-12-01,[https://open.overheid.nl/documenten/ronl-1534...,2021-12-03,Ministerie van Financiën,363
24,Beslissing op bezwaar op besluit Wob over VOC-...,2022-10-17,[https://open.overheid.nl/documenten/ronl-201b...,2021-09-10,"Ministerie van Landbouw, Natuur en Voedselkwal...",402
27,Beslissing op bezwaar tegen Woo-besluit over A...,2023-02-02,[https://open.overheid.nl/documenten/ronl-f944...,2022-08-09,Ministerie van Buitenlandse Zaken,177
...,...,...,...,...,...,...
1124,Besluit over het corrigeren van onvolkomenhede...,2023-01-05,[https://open.overheid.nl/documenten/ronl-29d6...,2022-10-14,"Ministerie van Landbouw, Natuur en Voedselkwal...",83
1129,Woo-verzoek Evaluatiecommissie Tuitjenhorn,2022-06-30,[https://open.overheid.nl/documenten/ronl-370d...,2021-10-22,Ministerie van Justitie en Veiligheid,251
1130,Woo-verzoek over de identiteit achter anonieme...,2022-08-25,[https://open.overheid.nl/documenten/ronl-2a0b...,2022-08-02,"Ministerie van Landbouw, Natuur en Voedselkwal...",23
1131,Woo-verzoek over landbouwvrijstelling in inkom...,2023-02-27,[https://open.overheid.nl/documenten/ronl-1c09...,2022-08-07,Ministerie van Financiën,204


De documentdatum van deze twee is gewoon volledig fout. En is de datum van het verzoek de documentdatum geworden terwijl het besluitdocument pas weken/maanden later uit is gekomen.

In [56]:
Data1 = df_dates.copy()
Data1 = Data1.drop_duplicates(subset = ['Verzoek'])
Data1 = Data1.groupby(['verantwoordelijke'])['verantwoordelijke'].count()

display(Data1)

# Data1.to_csv(r'C:\Users\ramon\Downloads\TestPlot.csv')

verantwoordelijke
Ministerie van Algemene Zaken                                 1
Ministerie van Binnenlandse Zaken en Koninkrijksrelaties     43
Ministerie van Buitenlandse Zaken                            33
Ministerie van Economische Zaken en Klimaat                  53
Ministerie van Financiën                                     80
Ministerie van Infrastructuur en Waterstaat                  35
Ministerie van Justitie en Veiligheid                        24
Ministerie van Landbouw, Natuur en Voedselkwaliteit         157
Ministerie van Onderwijs, Cultuur en Wetenschap              29
Ministerie van Sociale Zaken en Werkgelegenheid              32
Ministerie van Volksgezondheid, Welzijn en Sport             27
Name: verantwoordelijke, dtype: int64

In [57]:
import plotly.express as px

# simpel plotje die al met de huidige data te maken is

fig = px.bar(y=Data1.index, x=Data1.values, text_auto='1s',
            title="Aantal WOO/WOB verzoeken per Ministerie")
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.show()

In [62]:
df2 = df_dates.copy()
df2 = df2.groupby(['verantwoordelijke'])['dagen'].mean()

fig = px.bar(x=df2.index, y=df2.values)
fig.show()